## 1. Importação das bibliotecas

In [1]:
import heapq
import json
import math
import nltk
import os
import pandas as pd
import spacy
import string
import time

## 2. Pré-Processamento

### 2.1. Definição da função de Pré-processamento

In [2]:
def remove_non_ascii(texto):
    return "".join(c for c in texto if ord(c) < 128)

In [3]:
def remove_special_characters(texto):
    return "".join(c for c in texto if c.isalnum() or c.isspace())

In [4]:
def remove_digits(texto):
    return "".join(c for c in texto if not c.isdigit())

A função **preprocessamento** faz a limpeza no texto removendo todos os caracteres que devem ser desconsiderados no processo de geração de tokens de sentenças e palavras.

In [5]:
def preprocessamento(texto):
    
    # Lista de palavras irrelevantes para a etapa de processamento do texto.
    stopwords = nltk.corpus.stopwords.words('english')

    # Lista de sinais de pontuação
    punctuation = string.punctuation

    texto = texto.lower()
    texto = remove_non_ascii(texto)
    texto = remove_special_characters(texto)
    texto = remove_digits(texto)
    
    #documento = pln(texto)
    
    tokens = []
    
    # Pré-processamento sem a lematização
    for token in nltk.word_tokenize(texto):
        tokens.append(token)
    
    # Pré-processamento com a lematização
    #for token in documento:
        #tokens.append(token.lemma_)

    # Filtrar apenas as palavras que não estiverem na lista de stopwords e nem na lista de pontuações
    tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in punctuation]
    
    # Converter a lista de palavras para uma String, não adiciona valores numéricos
    texto = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

    return texto

### 2.2. Definição das funções de tokenização

A função **token_sentences** transforma cada sentença do texto original em um token. Cada frase do texto, é considerado um token.

In [6]:
# Transforma cada frase do texto original em um token.
def token_sentences(texto):
    sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
    
    return sentencas_originais

A função **preprocessing_sentences** recebe uma lista de sentenças com o texto original e executa a função de preprocessamento, eliminando caracteres especiais, sinais de pontuação, dígitos, stop words, etc. Essa função retorna uma lista de sentenças formatadas (pré-processadas)

In [7]:
# Pré-processamento das sentenças originais
def preprocessing_sentences(sentencas_originais):
    sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
    
    return sentencas_formatadas

## 3. Algoritmo de Luhn

### 3.1. Definição da função important_words

A função **important_words** recebe uma lista de sentenças pré-processadas e gera uma lista com as palavras mais importantes do texto. É importante observar que não podemos considerar uma quantidade fixa de palavras importantes. O tamanho do texto e q quantidade de palavras distintas que ele possui interferem na quantidade de palavras importantes que devemos selecionar. 

Este algoritmo considera que número de palavras importantes corresponde a 10% do número total de palavras distintas existentes no texto. 

Observação: Esse cálculo de 10% é um parâmetro que pode ser ajustado na etapa dos testes.

In [8]:
def important_words(sentencas_formatadas):
    
    # Transforma cada palavra da sentença formatada em um token
    palavras = [palavra for sentenca in sentencas_formatadas for palavra in nltk.word_tokenize(sentenca)]
    
    # Calcula a frequência de cada palavra
    frequencia = nltk.FreqDist(palavras)
    
    # Considerar 30% das palavras como as mais importantes
    total_palavras = len(frequencia)
    n_top_palavras = round(0.3 * total_palavras)
    
    # Lista com as n palavras com maior frequência no texto
    top_n_palavras = [palavra[0] for palavra in frequencia.most_common(n_top_palavras)]
    
    return top_n_palavras

### 3.2. Definição da função sentence_grade

A função **sentence_grade** calcula a nota de cada sentença baseada na frequência de palavras importantes. As sentenças de melhores notas serão selecionadas para fazerem parte do sumário do texto. Essa função retorna uma lista e cada elemento contém o índice da sentença e a nota.

A nota da sentença pode ser calculada pela seguinte fórmula:

$$ 
Grade = \frac{IW^2}{GW} \\
$$

$$
IW: Important\ Words \\
GW: Total\ words\ in\ group
$$

In [9]:
def sentence_grade(sentencas, palavras_importantes, distancia):
    notas = []
    indice_sentenca = 0

    # Percorre cada uma das sentenças que passamos como parâmetro e gera uma lista de tokens (palavras)
    for sentenca in [nltk.word_tokenize(sentenca.lower()) for sentenca in sentencas]:
        indice_palavra = []
        
        # Percorre a lista de palavras importantes para verificar em qual posição 
        # de cada sentença essas palavras estão localizadas
        for palavra in palavras_importantes:
            try:
                # Índice onde cada uma das palavras importantes está armazenada na sentença.
                indice_palavra.append(sentenca.index(palavra))
            # Nem todas as sentenças possuem as palavras importantes. 
            # A exceção é para quando a palavra não estiver na sentença.
            except ValueError:
                pass
        
        # Ordenando a lista de índices de ocorrências de palavras importantes na sentença
        indice_palavra.sort()

        # Sentenças que não possuem palavras importantes serão ignoradas.
        if len(indice_palavra) == 0:
            continue

        # Quanto mais palavras importantes dentro de um grupo, mais importante é a sentença para o texto
        # [0, 1, 3, 5]
        lista_grupos = []
        grupo = [indice_palavra[0]]
        i = 1
        while i < len(indice_palavra):
            # Verifica a distância entre as palavras importantes
            if indice_palavra[i] - indice_palavra[i - 1] < distancia:
                grupo.append(indice_palavra[i])
            else:
                lista_grupos.append(grupo[:])
                grupo = [indice_palavra[i]]
            i += 1
        lista_grupos.append(grupo)

        nota_maxima_grupo = 0
        for g in lista_grupos:
            palavras_importantes_no_grupo = len(g)
            total_palavras_no_grupo = g[-1] - g[0] + 1
            # Fórmula para calcular a nota da sentença
            nota = 1.0 * palavras_importantes_no_grupo**2 / total_palavras_no_grupo

            # Cálculo para a maior nota do grupo
            if nota > nota_maxima_grupo:
                nota_maxima_grupo = nota

        # notas: lista com dois elementos: nota_max_grupo e índice da sentença
        notas.append((nota_maxima_grupo, indice_sentenca))
        indice_sentenca += 1

    return notas

### 3.3. Definição da função summarize

A função **summarize** executa os seguintes passos:
- Calcula a nota de cada sentença
- Gera uma lista com os índices e notas das melhores sentenças.
- Gera uma lista com as melhores sentenças descritas no texto original, ou seja, sem as exclusões realizadas na etapa do pré-processamento.

Os seguintes parâmetros são passados para a função:
- sentencas_originais: lista de tokens (sentenças) com o texto original;
- sentencas_formatadas: lista de tokens (sentenças) pré-processadas;
- top_n_palavras: lista com as palavras mais importantes do texto
- distancia: distância a ser considerada entre as palavras imporantes dentro de um mesmo grupo (valor parametrizável)
- quantidade_setencas: Número de sentenças que irão compor o resumo do texto.

Observação: Este algoritmo considera que a quantidade de sentenças que irão compor o resumo corresponde a 40% do número total de sentenças do texto. (Esse valor pode ser alterado na etapa dos testes)

In [10]:
def summarize(sentencas_originais, sentencas_formatadas, top_n_palavras, distancia, quantidade_sentencas):
    
    # obtem as notas de cada sentença chamando a função calcula_nota_sentenca
    notas_sentencas = sentence_grade(sentencas_formatadas, top_n_palavras, distancia)
    
    # Gera uma lista com os índices e notas das melhores sentenças
    lista_indices_melhores_sentencas = []
    notas_melhores_sentencas = heapq.nlargest(quantidade_sentencas, notas_sentencas)
    for tupla in notas_melhores_sentencas:
        lista_indices_melhores_sentencas.append(tupla[1])
    lista_indices_melhores_sentencas.sort()
    
    # Cria a lista de melhores_sentencas com os textos originais
    melhores_sentencas = [sentencas_originais[i] for i in lista_indices_melhores_sentencas]
    
    return sentencas_originais, melhores_sentencas, notas_sentencas

### 3.4. Definição da função view_summary

A função **view_summary** exibe o sumário do texto resultante após a aplicação do algoritmo de sumarização. Esta função retorna as melhores sentenças com o texto original e concatenadas. 

In [11]:
def view_summary(lista_sentencas, melhores_sentencas):
    texto = ''
    
    for sentenca in lista_sentencas:
        if sentenca in melhores_sentencas:
            texto = texto + ' ' + sentenca
    return texto

### 3.5. Execução do Algoritmo para todas as notícias

Esse trecho de código corresponde à execução completa do **Algoritmo de Luhn** para um lote de notícias. Os seguintes passos são executados:



In [12]:
# Leitura do arquivo JSON com a base de dados de notícias
dt = pd.read_json("news.json")
display(dt)

,titulo,conteudo,sumario
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,"From her special numbers to TVappearances, Bol...",Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...,...
3960,Rasna seeking ?250 cr revenue from snack categ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",Fruit juice concentrate maker Rasna is eyeing ...
3961,Sachin attends Rajya Sabha after questions on ...,Former cricketer Sachin Tendulkar was spotted ...,Former Indian cricketer Sachin Tendulkar atten...
3962,Shouldn't rob their childhood: Aamir on kids r...,"Aamir Khan, whose last film Dangal told the st...","Aamir Khan, while talking about reality shows ..."
3963,"Asha Bhosle gets ?53,000 power bill for unused...",Maharahstra Power Minister Chandrashekhar Bawa...,The Maharashtra government has initiated an in...


In [13]:
# Lista para armazenar o resultado do processamento do Algoritmo Luhn para todas as notícias
algoritmo_luhn = []
processamento_luhn = []

inicio = time.time()
processamento_luhn.append({'Begin': inicio})
print('Begin:', inicio)

for i in range(len(dt)):
    titulo = dt.iloc[i, 0]
    conteudo = dt.iloc[i, 1]
    sumario = dt.iloc[i, 2]
    
    # Geração de tokens (sentenças) com os textos originais
    sentencas_originais = token_sentences(conteudo)
    
    # Executar o pre-processamento das sentenças para eliminar os caracteres e palavras irrelevantes
    sentencas_formatadas = preprocessing_sentences(sentencas_originais)
    
    # Gerar a lista das n palavras mais importantes do texto
    top_n_palavras = important_words(sentencas_formatadas)
    
    # Quantidade de sentenças que irão compor o sumário: 20% do total de sentenças
    quantidade_sentencas = math.ceil(len(sentencas_originais)*0.2)
    
    # Distância entre as palavras importantes no grupo
    distancia = 3
    
    # Geração das melhores sentenças que irão compor o sumário do texto
    sentencas_originais, melhores_sentencas, notas_sentencas = summarize(sentencas_originais, sentencas_formatadas, top_n_palavras, distancia, quantidade_sentencas)
    
    # Geração do sumário somente com as frases correspondentes às melhores sentenças do texto
    sumario_luhn = view_summary(sentencas_originais, melhores_sentencas)
    
    algoritmo_luhn.append({'index': i, 'titulo': titulo, 'conteudo': conteudo, 'sumario': sumario, 'sumario_luhn': sumario_luhn})
    if i % 100 == 0:
        print(i)

fim = time.time()
print('Fim da execução:', fim)
processamento_luhn.append({'End': fim})

duracao = fim - inicio
print('Duração:', duracao)
processamento_luhn.append({'Duration': duracao})

Begin: 1635870314.8275368
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
Fim da execução: 1635870379.4650922
Duração: 64.63755536079407


In [14]:
# Será criado um arquivo algoritmo_luhn.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('algoritmo_luhn.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(algoritmo_luhn, indent=1))
arquivo.close()

In [15]:
# Será criado um arquivo processamento_luhn.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('processamento_luhn.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(processamento_luhn, indent=1))
arquivo.close()